In [1]:
import os
import json
from conditional_rate_matching import data_path
import pretty_midi
import librosa

from matplotlib import pyplot as plt

raw_data = os.path.join(data_path,"raw")
#DATA_PATH = os.path.join(data_path,"raw","lmd_matched")

In [34]:
mid_file_path = os.path.join(raw_data,'lmd_full','lmd_full','0','00000ec8a66b6bd2ef809b0443eeae41.mid')

In [26]:
midi_data = pretty_midi.PrettyMIDI(mid_file)

In [36]:
is_valid_midi(mid_file_path, max_note_duration=50)

(False, 'A note is held for more than the allowed duration.')

In [33]:

def is_valid_midi(file_path, max_note_duration=50, total_time_steps=256):
    # Load the MIDI file
    midi_data = pretty_midi.PrettyMIDI(file_path)

    # Assuming we're only dealing with the first instrument
    instrument = midi_data.instruments[0]

    # Sort notes by start time
    notes = sorted(instrument.notes, key=lambda note: note.start)

    # Check for monophonic
    for i in range(len(notes) - 1):
        if notes[i].end > notes[i + 1].start:
            return False, "The track is not monophonic (notes overlap)."

    # Check for more than one type of note
    unique_notes = set([note.pitch for note in notes])
    if len(unique_notes) <= 1:
        return False, "The track does not have more than one type of note."

    # Check for note duration
    for note in notes:
        duration = midi_data.time_to_tick(note.end) - midi_data.time_to_tick(note.start)
        if duration > max_note_duration:
            return False, "A note is held for more than the allowed duration."

    return True, "The MIDI file meets all criteria."

In [113]:
piano_roll.max()

90.0

In [ ]:
import os
import pretty_midi

def find_midi_files(root_folder, limit=None):
    midi_files = []
    for subdir, dirs, files in os.walk(root_folder):
        for file in files:
            if file.lower().endswith(('.mid', '.midi')):
                midi_files.append(os.path.join(subdir, file))
                if limit is not None and len(midi_files) >= limit:
                    return midi_files
    return midi_files

def process_midi_files(midi_files):
    for file_path in midi_files:
        # Processing each MIDI file
        print(f"Processing {file_path}")
        valid, message = is_valid_midi(file_path)  # Assuming is_valid_midi is defined
        print(message)

# Example usage
root_folder = 'path_to_your_root_folder'
file_limit = 10  # Limit the number of files to process
midi_files = find_midi_files(root_folder, limit=file_limit)
process_midi_files(midi_files)
